# Floodplain Mapping via RasProcess CLI (Recommended for Windows)

This notebook demonstrates floodplain mapping using **RasProcess.exe**, the fastest and most reliable method for Windows users.

## When to Use This Method

✅ **Best for:**
- Windows environments
- Batch processing multiple plans
- Production workflows requiring speed
- All map variables (WSE, Depth, Velocity, Froude, Shear, D*V, D*V²)
- Time-series analysis (specific timesteps)

❌ **Not suitable for:**
- Cloud/Docker environments (requires HEC-RAS installation)
- Linux/Mac environments (Windows-only)

## Comparison with Other Methods

| Method | Speed | Reliability | Matches HEC-RAS to 0.01' | Cloud-Compatible | GUI Required | Variables |
|--------|-------|-------------|--------------------------|------------------|--------------|-----------|
| **15_b: RasProcess CLI** | ⭐⭐⭐ Fastest (8-10 sec) | ⭐⭐⭐ Excellent | ✅ Yes (100% - native) | ❌ No | ❌ No | All |
| 15_c: Python-GIS | ⭐⭐ Moderate (15-20 sec) | ⭐⭐⭐ Excellent | ✅ Yes (horizontal)* | ✅ Yes | ❌ No | WSE, Depth, Velocity (2D only) |
| 15_a: GUI Automation | ⭐ Slow (60+ sec) | ⭐ Fragile | ✅ Yes (100% - native) | ❌ No | ✅ Yes | All |

\* Python-GIS matches HEC-RAS to 0.01' for **2D horizontal interpolation only** (validation tests in notebook 15_c). Limitations: 2D mesh only (no 1D support yet), horizontal interpolation only.

**Recommendation**: Use this method (15_b) for Windows-based workflows.

## Prerequisites

- HEC-RAS 6.x installed (provides RasProcess.exe)
- Windows operating system
- Computed HEC-RAS plan with HDF results
- rasterio (optional, for georeferencing fix)

In [ ]:
#!pip install --upgrade ras-commander

In [ ]:
#!pip uninstall -y ras-commander

In [ ]:
# Import ras-commander
from pathlib import Path
import sys

# Flexible import for development vs installed package
try:
    from ras_commander import *
    from ras_commander import RasMap
    print("ras-commander pip package loaded")
except ImportError:
    current_file = Path.cwd()
    parent_directory = current_file.parent
    sys.path.insert(0, str(parent_directory))
    from ras_commander import *
    print("ras-commander local copy loaded")

In [ ]:
# Additional imports for visualization
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

## Step 1: Initialize Project

We'll use the BaldEagleCrkMulti2D example project.

In [ ]:
# Extract example project
project_path = RasExamples.extract_project("BaldEagleCrkMulti2D")

# Initialize project
init_ras_project(project_path, "6.6")

print(f"Project: {ras.project_name}")
print(f"Folder: {ras.project_folder}")

## Step 2: Pre-requisite - Check .rasmap Compatibility

**IMPORTANT**: For HEC-RAS 5.0.7 projects running in 6.x, the `.rasmap` file must be upgraded to 6.x format before stored map generation will work.

The `ensure_rasmap_compatible()` function automatically detects and upgrades old `.rasmap` files.

In [ ]:
# Check .rasmap compatibility (upgrade if needed)
result = RasMap.ensure_rasmap_compatible(auto_upgrade=True)

print(f"Status: {result['status']}")
print(f"Message: {result['message']}")
print(f"Version: {result['version']}")

if result['status'] == 'manual_needed':
    print("\n⚠️ Manual intervention required:")
    print("1. Open project in HEC-RAS")
    print("2. Click 'GIS Tools' > 'RAS Mapper'")
    print("3. Wait for RASMapper to open (this upgrades .rasmap)")
    print("4. Close RASMapper and HEC-RAS")
    print("5. Re-run this notebook")
else:
    print("\n✅ .rasmap file is ready for stored map generation")

## Step 3: Compute Plan (if needed)

Make sure plan 06 has been computed.

In [ ]:
# Check if plan 06 has HDF results
hdf_path = ras.project_folder / f"{ras.project_name}.p06.hdf"

if not hdf_path.exists():
    print("Computing plan 06...")
    RasCmdr.compute_plan("06")
else:
    print("Plan 06 already computed")

## Step 4: Generate Floodplain Maps

### Available Map Types

| Parameter | Map Type | Description |
|-----------|----------|-------------|
| `wse` | Water Surface Elevation | Maximum WSE |
| `depth` | Water Depth | Maximum depth |
| `velocity` | Velocity Magnitude | Maximum velocity |
| `froude` | Froude Number | Flow regime indicator |
| `shear_stress` | Shear Stress | Bed shear stress |
| `depth_x_velocity` | D*V | Hazard metric |
| `depth_x_velocity_sq` | D*V² | Impact metric |

In [ ]:
#!pip install --upgrade ras-commander

In [ ]:
# Generate WSE, Depth, Velocity maps for plan 06
print("Generating stored maps for Plan 06...")
print("This may take 1-2 minutes.\n")

results = RasProcess.store_maps(
    plan_number="06",
    profile="Max",
    wse=True,
    depth=True,
    depth_x_velocity=True
    velocity=True,
    fix_georef=True,
    ras_version="6.6"
)

print("\nGenerated files:")
for map_type, files in results.items():
    print(f"  {map_type}: {len(files)} file(s)")
    for f in files:
        print(f"    - {f.name}")

In [ ]:
# Stop script execution after this cell.
import sys
sys.exit("Script stopped here as requested.")


## Step 5: Generate Additional Variables

RasProcess supports all HEC-RAS output variables.

In [ ]:
# Generate Froude and D*V hazard maps
results_extra = RasProcess.store_maps(
    plan_number="06",
    profile="Max",
    wse=False,
    depth=False,
    velocity=False,
    froude=True,
    depth_x_velocity=True,
    fix_georef=True,
    ras_version="6.6"
)

print("Generated additional maps:")
for map_type, files in results_extra.items():
    for f in files:
        print(f"  - {f.name}")

## Step 6: Time-Series Maps

Generate maps for specific timesteps instead of Max/Min.

In [ ]:
# Get available timestamps
timestamps = RasProcess.get_plan_timestamps("06")

print(f"Available timestamps: {len(timestamps)} total")
print(f"  First: {timestamps[0]}")
print(f"  Last:  {timestamps[-1]}")

# Generate map for specific timestep
if len(timestamps) > 10:
    target_ts = timestamps[10]
    print(f"\nGenerating WSE for timestep: {target_ts}")
    
    results_ts = RasProcess.store_maps(
        plan_number="06",
        profile=target_ts,
        wse=True,
        depth=False,
        velocity=False,
        fix_georef=True,
        ras_version="6.6"
    )
    
    for f in results_ts['wse']:
        print(f"  Generated: {f.name}")

## Step 7: Visualize Results

In [ ]:
# Plot WSE map
if 'wse' in results and results['wse']:
    with rasterio.open(results['wse'][0]) as src:
        fig, ax = plt.subplots(figsize=(10, 8))
        show(src, ax=ax, cmap='terrain', title='Maximum Water Surface Elevation')
        plt.tight_layout()
        plt.show()
        
        print(f"CRS: {src.crs}")
        print(f"Resolution: {src.res}")
        print(f"Bounds: {src.bounds}")

## Step 8: Batch Processing

Generate maps for all plans with HDF results.

In [ ]:
# Generate maps for all plans
all_results = RasProcess.store_all_maps(
    profile="Max",
    wse=True,
    depth=True,
    velocity=True,
    fix_georef=True,
    ras_version="6.6"
)

print("\nBatch Processing Results:")
print("=" * 60)
for plan_num, files in all_results.items():
    if 'error' in files:
        print(f"Plan {plan_num}: ERROR - {files['error']}")
    else:
        total = sum(len(f) for f in files.values())
        print(f"Plan {plan_num}: {total} files generated")

## Technical Explanation: How RasProcess Works

### RasProcess.exe CLI Tool

RasProcess.exe is an **undocumented command-line utility** bundled with HEC-RAS 6.x that enables headless map generation without opening the RASMapper GUI.

**Location**: `C:\Program Files (x86)\HEC\HEC-RAS\6.6\RasProcess.exe`

### Workflow

1. **Modify .rasmap XML**: Add `<Layer>` elements for desired map types
   ```xml
   <Layer Name="WSE" Type="RASResultsMap" Filename=".\Results\WSE (Max).vrt">
     <MapParameters MapType="elevation" ProfileName="Max"/>
   </Layer>
   ```

2. **Execute RasProcess.exe**: Run subprocess with XML command
   ```bash
   RasProcess.exe -Command=StoreAllMaps \
                  -RasMapFilename=project.rasmap \
                  -ResultFilename=project.p06.hdf
   ```

3. **Parse Output**: Scan folder for generated .tif files

4. **Fix Georeferencing**: Apply CRS from project projection file
   - HEC-RAS outputs TIFs without proper CRS metadata
   - `fix_georef=True` uses rasterio to add CRS

5. **Restore .rasmap**: Replace modified .rasmap with backup

### Performance

RasProcess is the **fastest method** because:
- Uses HEC-RAS native rasterization engine (same as RASMapper)
- No GUI overhead
- Optimized C++ code
- Parallel processing within HEC-RAS

**Typical performance** (BaldEagleCrkMulti2D, 3 variables):
- RasProcess CLI: 8-10 seconds
- Python-GIS: 15-20 seconds  
- GUI Automation: 60+ seconds

## Troubleshooting

### Issue: RasProcess.exe not found
**Solution**: Ensure HEC-RAS 6.x is installed. Check path manually:
```python
rasprocess_path = RasProcess.find_rasprocess("6.6")
print(rasprocess_path)
```

### Issue: No maps generated
**Solution**: 
1. Check .rasmap upgrade: `RasMap.ensure_rasmap_compatible()`
2. Verify HDF exists: `(ras.project_folder / f"{ras.project_name}.p06.hdf").exists()`
3. Check plan has results: `HdfPlan.get_plan_info(hdf_path)`

### Issue: GDAL NoDataValue warning
**Solution**: This warning is cosmetic and can be ignored. It's a GDAL issue with HEC-RAS TIF format.

### Issue: Missing CRS in output
**Solution**: Use `fix_georef=True` parameter to apply CRS from project.

## Decision Matrix: When to Use Each Method

### Use 15_b (RasProcess CLI) when:
- ✅ Running on Windows
- ✅ Need fastest performance
- ✅ Need all map variables (Froude, Shear, D*V, etc.)
- ✅ Batch processing multiple plans
- ✅ Production workflows

### Use 15_c (Python-GIS) when:
- ✅ Running in cloud/Docker
- ✅ Linux/Mac environment
- ✅ Only need WSE, Depth, Velocity
- ✅ 2D mesh projects only
- ✅ Reproducible research workflows

### Use 15_a (GUI Automation) when:
- ✅ Need visual feedback
- ✅ Debugging map generation issues
- ⚠️ Last resort (most fragile method)

## Summary

This notebook demonstrated:

1. ✅ `.rasmap` compatibility checking and auto-upgrade
2. ✅ Individual map generation with `store_maps()`
3. ✅ All map variables (WSE, Depth, Velocity, Froude, etc.)
4. ✅ Time-series analysis with specific timesteps
5. ✅ Batch processing with `store_all_maps()`
6. ✅ Visualization with rasterio
7. ✅ Technical explanation of RasProcess.exe workflow

**Next Steps**:
- See notebook 15_c for cloud-compatible Python-GIS method
- See notebook 15_a for GUI automation method
- See notebook 11 for detailed HDF data extraction